In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
data = pd.read_excel('ID.xls')
data.head()

,День месяца,Время вылета,Связка,Назначение,Тип судна,Тип связи,Налет,Экипаж
0,1,02:55,route 1,city 61,А-320,ВВЛ,10:50,4
1,1,03:55,route 2,city 60,А-320,МВЛ,06:00,4
2,1,04:10,route 3,city 23,А-319,ВВЛ,03:00,3
3,1,04:40,route 4,city 32,А-319,МВЛ,08:35,3
4,1,05:00,route 5,city 8,А-319,ВВЛ,06:15,3


In [3]:
solution = pd.read_excel('Expert.xls')
solution.head()

,Связка,Вылет,Тип,1,2,3,4,5,6,7,...,22,23,24,25,26,27,28,29,30,31
0,NaN,NaN,ВС,СБ,ВС,ПН,ВТ,СР,ЧТ,ПТ,...,СБ,ВС,ПН,ВТ,СР,ЧТ,ПТ,СБ,ВС,ПН
1,route 8,05:10,А 320,2,2,5,2,2,3,1,...,6,4,2,6,6,5,2,5,5,3
2,route 28,11:20,А 319,6,4,6,1,5,5,3,...,6,4,1,2,3,4,5,5,5,2
3,route 66,22:45,А 319,5,3,1,4,3,2,1,...,4,3,5,6,4,1,2,3,4,6
4,route 31,11:55,А 319,NaN,NaN,4,3,4,1,5,...,NaN,NaN,4,5,1,2,4,NaN,NaN,1


In [4]:
flight = data.iloc[0]
flight

День месяца           1
Время вылета      02:55
Связка          route 1
Назначение      city 61
Тип судна        А-320 
Тип связи           ВВЛ
Налет             10:50
Экипаж                4
Name: 0, dtype: object

In [5]:
night_start = '22:00'
night_end = '6:00'

In [6]:
start = flight['Время вылета']
duration = flight['Налет']

In [7]:
def sum_time(start, duration):
    st = list(map(lambda x: int(x), start.split(':')))
    dur = list(map(lambda x: int(x), duration.split(':')))
    a = [st[0] + dur[0],st[1] + dur[1]]
    if a[1]>= 60:
        a[0] += 1
        a[1] -= 60
    if a[0]>=24:
        a[0] -= 24
    return str(a[0])+':'+str(a[1])

In [8]:
def in_minuts(duration):
    dur  = np.array(list(map(lambda x: int(x), duration.split(':'))))
    return dur[0]*60 + dur[1]

In [9]:
def in_hours(minuts):
    h = int(minuts // 60)
    m = int(minuts % 60)
    if h < 10:
        h = '0'+ str(h)
    else:
        h = str(h)
    if m < 10:
        m = '0'+ str(m)
    else:
        m = str(m)
    return h + ':' + m

In [10]:
def find_night_minuts(fligt_time):
    night_h1 = list(map(lambda x: in_minuts(x), ['00:00','06:00']))
    night_h2 = list(map(lambda x: in_minuts(x), ['22:00','23:59']))
    f = list(map(lambda x: in_minuts(x), fligt_time))
    delta = 0

    if f[0] < night_h1[1]:
        if f[1] < night_h1[1]: # весь полёт попал в первую половину ночи
            delta += f[1] - f[0] # Длительность полёта в минутах
        else:
            delta += 6*60 - f[0] 
    elif f[0] > night_h1[1] and f[0] < night_h2[0]:
        if f[1] > night_h1[1] and f[1] < night_h2[0]:
            if f[0] < f[1]:
                delta += 0  #дневное время
            else:
                delta += night_h1[1] + night_h2[1] - night_h2[0] + 1#очень длинный перелёт

        elif f[1] > night_h2[0] and f[1] < night_h2[1]:
            delta += f[1] - night_h2[0] #часть полёта в вечернее время

        elif f[1] > night_h1[0] and f[1] < night_h1[1]:
            delta += f[1] + night_h2[1] - night_h2[0] + 1

    elif f[0] > night_h2[0]:
        if f[1] > night_h2[0] and f[1] < night_h2[1]:
            delta += f[1]-f[0]
        elif f[1] < night_h1[1]: 
            delta += night_h2[1] - f[0] + f[1] + 1
        elif f[1] > night_h1[1] and f[1] < night_h2[0]:
            delta += night_h2[1] - f[0] + night_h1[1] + 1

    return delta    

In [11]:
def night_percent(start, duration): # start и duration 'hh:mm'
    d0 = in_minuts(duration)
    d = in_hours(d0/2)
    t = sum_time(start, d)
    h1 = [start, t]
    t = sum_time(t, '01:30')
    h2 = [t, sum_time(t,d)]
    return (find_night_minuts(h1) + find_night_minuts(h2))/d0

In [12]:
def is_night_flight(flight):
    start = flight['Время вылета']
    duration = flight['Налет']
    n_pc = night_percent(start, duration)
    if n_pc > 0.5:
        return True
    else:
        return False

In [13]:
is_night_flight(flight)

False